.. _nb_repair:

## Repair Operator

The repair operator is mostly problem dependent. Most commonly it is used to make sure the algorithm is only searching in the feasible space. It is applied after the offsprings have been reproduced. In the following, we are using the knapsack problem to demonstrate the repair operator in *pymoo*.



In the well-known **Knapsack Problem**. In this problem, a knapsack has to be filled with items without violating the maximum weight constraint. Each item $j$ has a value $b_j \geq 0$  and a weight $w_j \geq 0$ where $j \in \{1, .., m\}$. The binary decision vector $z = (z_1, .., z_m)$ defines, if an item is picked or not. The aim is to maximize the profit $g(z)$:

\begin{eqnarray}
max & & g(z) \\[2mm] \notag 
\text{s.t.} & & \sum_{j=1}^m z_j \, w_j \leq Q \\[1mm] \notag 
& & z = (z_1, .., z_m) \in \mathbb{B}^m \\[1mm] \notag 
g(z) & = & \sum_{j=1}^{m}  z_j \, b_j \\[2mm] \notag 
\end{eqnarray}


A simple GA will have some infeasible evluations in the beginning and then concentrate on the infeasible space.


In [1]:
from pymoo.factory import get_algorithm, get_crossover, get_mutation, get_sampling
from pymoo.optimize import minimize
from pymoo.problems.single.knapsack import create_random_knapsack_problem

problem = create_random_knapsack_problem(30)

algorithm = get_algorithm("ga",
                       pop_size=200,
                       sampling=get_sampling("bin_random"),
                       crossover=get_crossover("bin_hux"),
                       mutation=get_mutation("bin_bitflip"),
                       elimate_duplicates=True)

res = minimize(problem,
               algorithm,
               termination=('n_gen', 10),
               verbose=True)

n_gen | n_eval  | cv (min/avg)                        | favg  | fopt 
1     | 200     | 2.3600000000E+02 / 5.1851000000E+02 | -     | -    
2     | 400     | 8.4000000000E+01 / 3.8401000000E+02 | -     | -    
3     | 600     | 7.6000000000E+01 / 2.8355000000E+02 | -     | -    
4     | 800     | 1.0000000000E+01 / 2.0667000000E+02 | -     | -    
5     | 1000    | 0.00000000000000 / 1.3477000000E+02 | -344.50000000000000 | -355.00000000000000
6     | 1200    | 0.00000000000000 / 8.2935000000E+01 | -274.69230769230768 | -415.00000000000000
7     | 1400    | 0.00000000000000 / 4.4475000000E+01 | -272.95238095238096 | -478.00000000000000
8     | 1600    | 0.00000000000000 / 1.3360000000E+01 | -283.22916666666669 | -503.00000000000000
9     | 1800    | 0.00000000000000 / 0.19500000000000 | -277.66489361702128 | -516.00000000000000
10    | 2000    | 0.00000000000000 / 0.00000000000000 | -338.85500000000002 | -537.00000000000000



Because the constraint $\sum_{j=1}^m z_j \, w_j \leq Q$ is fairly easy to satisfy. Therefore, we can make sure before evaluating the objective function, that this constraint is not violated by repairing the individual.
A repair class has to be defined and the population are given as input. The repaired population has to be returned.

In [2]:
import numpy as np
from pymoo.model.repair import Repair


class ConsiderMaximumWeightRepair(Repair):

    def _do(self, problem, pop, **kwargs):
        
        # maximum capacity for the problem
        Q = problem.C
        
        # the packing plan for the whole population (each row one individual)
        Z = pop.get("X")
        
        # the corresponding weight of each individual
        weights = (Z * problem.W).sum(axis=1)
        
        # now repair each indvidiual i
        for i in range(len(Z)):
            
            # the packing plan for i
            z = Z[i]
            
            # while the maximum capacity violation holds
            while weights[i] > Q:
                
                # randomly select an item currently picked
                item_to_remove = np.random.choice(np.where(z)[0])
                
                # and remove it
                z[item_to_remove] = False
                
                # adjust the weight
                weights[i] -= problem.W[item_to_remove]
          
        # set the design variables for the population
        pop.set("X", Z)
        return pop

In [3]:
algorithm.repair = ConsiderMaximumWeightRepair()

res = minimize(problem,
               algorithm,
               termination=('n_gen', 10),
               verbose=True)


n_gen | n_eval  | cv (min/avg)                        | favg             | fopt            
1     | 200     | 0.00000000000000 / 0.00000000000000 | -127.09000000000000 | -396.00000000000000
2     | 400     | 0.00000000000000 / 0.00000000000000 | -225.65000000000001 | -510.00000000000000
3     | 600     | 0.00000000000000 / 0.00000000000000 | -291.05500000000001 | -510.00000000000000
4     | 800     | 0.00000000000000 / 0.00000000000000 | -342.62000000000000 | -555.00000000000000
5     | 1000    | 0.00000000000000 / 0.00000000000000 | -386.74500000000000 | -579.00000000000000
6     | 1200    | 0.00000000000000 / 0.00000000000000 | -424.20999999999998 | -579.00000000000000
7     | 1400    | 0.00000000000000 / 0.00000000000000 | -456.69999999999999 | -640.00000000000000


8     | 1600    | 0.00000000000000 / 0.00000000000000 | -492.25000000000000 | -670.00000000000000
9     | 1800    | 0.00000000000000 / 0.00000000000000 | -518.85000000000002 | -670.00000000000000
10    | 2000    | 0.00000000000000 / 0.00000000000000 | -541.68499999999995 | -670.00000000000000


As it can be seen, the repair operator makes sure no infeasible solution is evaluated. Even though this exapmle seems to be quite easy, the repair operator makes especially sense for more complex constraints where domain specific knowledge is known.